## [Stable Marriage](https://www.cs.princeton.edu/courses/archive/spr05/cos423/lectures/01stable-matching.pdf)
### Problem
- Give:
    + N men
    + N women
- Each person has ranked all members of the opposite sex in **order of preference**. Example:
```python
# Men
    0: 8, 6, 7, 5, 9
    1: 9, 7, 6, 5, 8
    2: 6, 9, 5, 8, 7
    3: 9, 6, 8, 7, 5
    4: 8, 5, 6, 7, 9
# Women
    5: 3, 1, 4, 2, 0
    6: 1, 0, 3, 2, 4
    7: 0, 2, 4, 3, 1
    8: 3, 0, 2, 1, 4
    9: 1, 4, 0, 2, 3
```

- Find a **Stable Matching** = Perfect matching + Stability:
    - **Perfect matching**
        + Each man gets exactly one woman.
        + Each woman gets exactly one man.
    - **Stability**: no unstable pairs exist 
        - **Unstable pair** =
            + Exist a pair (m_, w_) which m_ **not marry** w_
            + m_ prefer w_ **and** w_ prefer m_ than their current partners

        - Example:
        ```python
        # Men
            0: 2, 3
            1: 2, 3
        # Women
            2: 0, 1
            3: 0, 1
        ```
            - Match {0,3}, {1,2} 
                + 0 prefer 2 over 3 and 2 prefer 0 over 1 -> **not stable**
            - Match {0,2}, {1,3} -> **stable**

### Gale–Shapley algorithm O($n^2$)
- A stable matching will **always exist**

```python
while exist_a_free_man():
    m = a_free_man()
    w = m_highest_reference()

    while m_not_yet_engaged():
        # Case: w is free
        if w_is_free():
            engage(m, w)
         
        # Case: w is already engaged with m*
        #       w prefer m than m*
        else if w_prefers_m_than_m*():
            divorce(m*, w)
            set_free(m*)

            engage(m, w)

        # Case: w is already engaged with m*
        #       w prefer m* than m
        else:
            w = next_m_preference()
```

### Time Complexity Analysis
- There are at most $n^2$ possible proposals

### Proof of Correctness
#### Termination
- Men propose to women in decreasing order of preference.
- Once a woman is matched, she never becomes unmatched, she only "trades up".

#### Perfection
- All men and women get engaged
- Proof by contradiction
    + Suppose a man m not engaged upon the termination of algorithm.
    + -> a woman w not engaged
    + -> w never got proposed (if a woman is proposed -> alway be engaged)
    + -> But m proposes to every woman if not engaged
    + -> Contrast

#### Stability
- No unstable pairs.
- Proof by contradiction
    + Suppose w* prefer m* and m* prefer w* over their current partners
    + Case 1: m* not propose to w* -> but m* propose in his priority -> m* prefer his current partner than w* -> constrast
    + Case 2: m* propose to w* -> but w* engage her partner -> w* prefer her partner than m* -> Contrast

In [1]:
import numpy as np
from queue import Queue
from collections import defaultdict


def solve_stable_marriage(preferences: np.ndarray) -> np.ndarray:
    '''
    Args:
        preferences: shape(2N, N)
          [0:N-1]: men[N]
          [N:2N-1]: women[N]

    Returns:
        married: shape(2N)
            married[m] = w, married[w] = m
    '''
    # N = number of men/women
    N:int = preferences.shape[0] // 2

    # Assert Input
    assert preferences.shape[0] == 2*preferences.shape[1]
    for m in range(N):
        check_size = set()
        for w in preferences[m]:
            assert N <= w and w < 2*N
            check_size.add(w)
        assert len(check_size) == N
    for w in range(N,2*N):
        check_size = set()
        for m in preferences[w]:
            assert 0 <= m and m < N
            check_size.add(m)
        assert len(check_size) == N

    # married[m] = w, married[w] = m
    married:np.ndarray = np.full((2*N,), -1)
    
    # Available men (not married) queue
    men_available = Queue()
    for m in range(N): men_available.put(m)
    
    # Denote: man(m), woman(w), and her husband(m_) :)
    # Check if women(w) prefer this man(m) than her husband(m_)
    #      priority = 0,1,2,...
    priority = defaultdict(lambda: defaultdict(int))
    for w in range(N, 2*N):
        for idx, m in enumerate(preferences[w]):
            priority[w][m] = idx
    def is_prefer(w:int, m:int, m_:int) -> bool:
        return priority[w][m] < priority[w][m_]

    # Start matching
    while not men_available.empty():
        # Pick an available man
        m = men_available.get()

        # Get him married, started with his most prefered woman
        for w in preferences[m]:
            # Her husband
            m_ = married[w]
            
            # Case 1: w is available -> engage (m, w)
            if m_ == -1:
                married[m] = w
                married[w] = m
                break
            
            # Case 2: w is already married with m_
            #         but w prefer m than m_
            elif is_prefer(w,m,m_):
                # Divorce (m_, w); Set m_ available
                married[m_] = -1
                men_available.put(m_)
                
                # Marry (m, w)
                married[m] = w
                married[w] = m
                break
            
            # Case 3: w is already married with m_
            #         w prefer m_ than m
            # Move to next lower prior w
    
    print('Married couples:', end=' ')
    for m_, w in enumerate(married[:N]): print(f'({m_} - {w})', end=' ')
    print()
    return married

In [2]:
def verify_stable_marriage(preferences:np.ndarray, married:np.ndarray) -> bool:
    '''
    Args:
        preferences: shape(2N, N)
          [0:N-1]: men[N]
          [N:2N-1]: women[N]

        married: shape(2N)
            married[m] = w, married[w] = m
    Returns:
        True if stable marriage condition meet
    '''
    # N = number of men/women
    N:int = preferences.shape[0] // 2

    # Assert input
    assert preferences.shape[0] == 2*preferences.shape[1]
    assert preferences.shape[0] == married.shape[0]
    for m in range(N):
        check_size = set()
        for w in preferences[m]:
            assert N <= w and w < 2*N
            check_size.add(w)
        assert len(check_size) == N
    for w in range(N,2*N):
        check_size = set()
        for m in preferences[w]:
            assert 0 <= m and m < N
            check_size.add(m)
        assert len(check_size) == N
    for person in range(2*N):
        partner = married[person]
        assert person == married[partner]
        if person < N: assert N <= partner and partner < 2*N
        else: assert 0 <= partner and partner < N
    

    # Generate possibly unstable pairs: (m_, w_) which m_ not marry w_
    unstable_pairs = [] 
    for m_ in range(N):
        for w_ in range(N, 2*N):
            if w_ != married[m_]: unstable_pairs.append( (m_, w_) )

    # Check if a person prefer a candidate than the current partner
    # priority = 0,1,2,...
    priority = defaultdict(lambda: defaultdict(int))
    for m in range(N):
        for idx, w in enumerate(preferences[m]):
            priority[m][w] = idx
    for w in range(N, 2*N):
        for idx, m in enumerate(preferences[w]):
            priority[w][m] = idx
    def is_prefer(who:int, candidate:int, current_partner:int) -> bool:
        return priority[who][candidate] < priority[who][current_partner]

    # Check
    for m_, w_ in unstable_pairs:
        # Her husband, his wife
        m = married[w_]
        w = married[m_]

        # Check if m_ prefer w_ than his wife w and w_ prefer m_ than her husband m
        if is_prefer(m_, w_, w) and is_prefer(w_, m_, m):
            print(f'Existing ({m_}, {w_}) unstable')
            return False
    return True

#### Testcase 1
 
```
2 3
2 3
0 1
0 1
```


In [3]:
preferences = np.array([
    [2, 3],
    [2, 3],
    [0, 1],
    [0, 1]])

married = solve_stable_marriage(preferences)
print('Correct: ', married) if verify_stable_marriage(preferences, married) else print('Not correct')

Married couples: (0 - 2) (1 - 3) 
Correct:  [2 3 0 1]


#### Test case 2

```
8 6 7 5 9
9 7 6 5 8
6 9 5 8 7
9 6 8 7 5
8 5 6 7 9
3 1 4 2 0
1 0 3 2 4
0 2 4 3 1
3 0 2 1 4
1 4 0 2 3
```


In [4]:
preferences = np.array([
    [8, 6, 7, 5, 9],
    [9, 7, 6, 5, 8],
    [6, 9, 5, 8, 7],
    [9, 6, 8, 7, 5],
    [8, 5, 6, 7, 9],
    [3, 1, 4, 2, 0],
    [1, 0, 3, 2, 4],
    [0, 2, 4, 3, 1],
    [3, 0, 2, 1, 4],
    [1, 4, 0, 2, 3]])

married = solve_stable_marriage(preferences)
print('Correct: ', married) if verify_stable_marriage(preferences, married) else print('Not correct')

Married couples: (0 - 8) (1 - 9) (2 - 7) (3 - 6) (4 - 5) 
Correct:  [8 9 7 6 5 4 3 2 0 1]


#### Test case 3

```
8 6 7 5 9
9 7 6 5 8
6 9 5 8 7
9 6 8 7 5
8 5 6 7 9
3 1 4 2 0
1 0 3 2 4
0 2 4 3 1
3 0 2 1 4
1 4 0 2 3
```


In [5]:
preferences = np.array([
    [7, 5, 6, 4],
    [5, 4, 6, 7],
    [4, 5, 6, 7],
    [4, 5, 6, 7],
    [0, 1, 2, 3],
    [0, 1, 2, 3],
    [0, 1, 2, 3],
    [0, 1, 2, 3]])

married = solve_stable_marriage(preferences)
print('Correct: ', married) if verify_stable_marriage(preferences, married) else print('Not correct')

Married couples: (0 - 7) (1 - 5) (2 - 4) (3 - 6) 
Correct:  [7 5 4 6 2 1 3 0]
